# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298310894227                   -0.85    5.2
  2   -8.300185762968       -2.73       -1.25    1.0
  3   -8.300445962513       -3.58       -1.89    3.5
  4   -8.300462365463       -4.79       -2.77    1.4
  5   -8.300464375819       -5.70       -3.10    2.9
  6   -8.300464492035       -6.93       -3.26    1.0
  7   -8.300464562266       -7.15       -3.40   11.6
  8   -8.300464601108       -7.41       -3.52    1.5
  9   -8.300464629564       -7.55       -3.69    1.0
 10   -8.300464640052       -7.98       -3.87    1.9
 11   -8.300464643754       -8.43       -4.29    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64191900304                   -0.70    6.5
  2   -16.67827960891       -1.44       -1.14    1.4
  3   -16.67922981416       -3.02       -1.88    3.9
  4   -16.67927950414       -4.30       -2.77    3.4
  5   -16.67928592884       -5.19       -3.14    5.4
  6   -16.67928620515       -6.56       -3.53    1.9
  7   -16.67928621939       -7.85       -4.03    1.2


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32674735978                   -0.56    7.1
  2   -33.33435662695       -2.12       -1.00    1.2
  3   -33.33600505205       -2.78       -1.72    4.5
  4   -33.33615996320       -3.81       -2.63    7.0
  5   -33.33685121433       -3.16       -2.35    7.5
  6   -33.33687665062       -4.59       -2.42    1.1
  7   -33.33694377178       -4.17       -3.62    3.0
  8   -33.33694385255       -7.09       -3.80    5.2
  9   -33.33694388267       -7.52       -4.02    2.6


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298544463852                   -0.85    5.6
  2   -8.300235701948       -2.77       -1.58    1.0
  3   -8.300219671281   +   -4.80       -2.05    7.9
  4   -8.300359706033       -3.85       -2.24    1.5
  5   -8.300464010771       -3.98       -3.43    1.0
  6   -8.300464552121       -6.27       -3.83    3.8
  7   -8.300464630578       -7.11       -4.19    1.8


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32614654521                   -0.56    7.6
  2   -33.32957807051       -2.46       -1.28    1.4
  3   -25.08657612972   +    0.92       -0.61    8.2
  4   -33.25680334343        0.91       -1.58    6.4
  5   -33.18701724312   +   -1.16       -1.31    3.9
  6   -32.39635453275   +   -0.10       -1.08    4.6
  7   -33.32643449822       -0.03       -1.93    4.4
  8   -33.33388296849       -2.13       -2.24    2.4
  9   -33.33534811502       -2.83       -2.37    2.4
 10   -33.33534187890   +   -5.21       -2.43    1.1
 11   -33.33626343998       -3.04       -2.59    1.6
 12   -33.33683780569       -3.24       -3.00    3.1
 13   -33.33692059308       -4.08       -3.28    3.1
 14   -33.33693396602       -4.87       -3.44    3.0
 15   -33.33693605975       -5.68       -3.59    4.4
 16   -33.33694281110       -5.17       -4.00    2.8
 17   -33.33694374552       -6.03       -4.35 

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.